# SnapTo3D Spatial‑Planning Pipeline

Flow: 
```
USER prompt -> LLM (scene‑graph JSON) -> Constraint Solver -> Blender CodeGen -> Render -> VLM Verifier
```

## 0  Setup and dependencies

In [ ]:
# ↳ Run once per environment
!source spatial-env/bin/activate
!pip install -r requirements.txt

## 1  Scene‑graph schema & helper classes

In [79]:
import helper
import os
from openai import OpenAI
import json
from dotenv import load_dotenv
from helper import save_scene_graph


load_dotenv()

ImportError: cannot import name 'save_scene_graph' from 'helper' (/Users/divinirakiza/Workspaces/CALTECH/cs159/snapTo3D/3D_craft/helper.py)

In [ ]:
import helper
import os
from openai import OpenAI
import json
from dotenv import load_dotenv
from helper import save_scene_graph


load_dotenv()

ImportError: cannot import name 'save_scene_graph' from 'helper' (/Users/divinirakiza/Workspaces/CALTECH/cs159/snapTo3D/3D_craft/helper.py)

## 2  LLM reasoning → JSON

In [54]:
MODEL = 'gpt-4'

OpenAPIClient = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm_to_scene(prompt: str) -> dict:
    """Call GPT with function‑calling and return a scene‑graph dict."""
    function = {
        'name': 'create_scene_graph',
        'parameters': {
            'type': 'object',
            'properties': {
                'nodes': {'type':'array', 'items':{'type':'object'}},
                'constraints': {'type':'array', 'items':{'type':'object'}},
                'room': {'type':'object'}
            },
            'required': ['nodes','constraints']
        }
    }
    
    system_message = """You are a spatial-planning assistant that creates scene graphs for 3D room layouts.
    Each node in the scene should have:
    - id: unique identifier
    - size: [width, depth, height] in meters
    - asset: name of the 3D model to use
    
    Each constraint should have:
    - type: 'left_of' or 'front_of'
    - a: id of first object
    - b: id of second object
    - gap: optional spacing in meters"""
    
    try:
        response = OpenAPIClient.chat.completions.create(
            model=MODEL,
            messages=[
                {'role':'system', 'content': system_message},
                {'role':'user', 'content': prompt}
            ],
            functions=[function],
            function_call={'name':'create_scene_graph'}
        )
        
        if not response.choices or not response.choices[0].message.function_call:
            raise ValueError("No valid response from API")
            
        print('response', response.choices[0].message)
        return json.loads(response.choices[0].message.function_call.arguments)
    except Exception as e:
        print(f"Error in llm_to_scene: {str(e)}")
        raise

## 3  Constraint solver → absolute coordinates

In [51]:
from ortools.sat.python import cp_model

GRID = 100 
from ortools.sat.python import cp_model

def solve_scene(scene: dict) -> dict:
    m = cp_model.CpModel()
    pos, half = {}, {}

    # centres (x,y) in 1 cm grid
    for n in scene['nodes']:
        w,d,_ = n['size']
        half[n['id']] = (int(w*GRID/2), int(d*GRID/2))
        pos[n['id']]  = (                       # centre coordinates
            m.NewIntVar(-500*GRID, 500*GRID, f'x_{n["id"]}'),
            m.NewIntVar(-500*GRID, 500*GRID, f'y_{n["id"]}')
        )

    # relational constraints --------------------------------------------------
    for c in scene['constraints']:
        a,b = c['a'], c['b']
        gap = int(c.get('gap', .3)*GRID)
        xa,ya = pos[a]; xb,yb = pos[b]
        wa,da = half[a]; wb,db = half[b]

        if c['type'] == 'left_of':
            m.Add(xa + wa + gap <= xb - wb)
        elif c['type'] == 'front_of':
            m.Add(ya + da + gap <= yb - db)
        # … add others …

    # NON-OVERLAP (boolean disjunction) ---------------------------------------
    ids = list(pos.keys())
    for i in range(len(ids)):
        for j in range(i+1, len(ids)):
            A, B = ids[i], ids[j]
            xa, ya = pos[A]; xb, yb = pos[B]
            wa, da = half[A]; wb, db = half[B]

            left   = m.NewBoolVar(f'{A}_left_{B}')
            right  = m.NewBoolVar(f'{A}_right_{B}')
            front  = m.NewBoolVar(f'{A}_front_{B}')
            behind = m.NewBoolVar(f'{A}_behind_{B}')

            m.Add(xa + wa <= xb - wb).OnlyEnforceIf(left)
            m.Add(xb + wb <= xa - wa).OnlyEnforceIf(right)
            m.Add(ya + da <= yb - db).OnlyEnforceIf(front)
            m.Add(yb + db <= ya - da).OnlyEnforceIf(behind)

            # at least one spatial separation must hold
            m.AddBoolOr([left, right, front, behind])

    # -------------------------------------------------------------------------
    solver = cp_model.CpSolver()
    assert solver.Solve(m) == cp_model.OPTIMAL, "No feasible layout found"
    return {oid: (solver.Value(x)/GRID, solver.Value(y)/GRID)
            for oid,(x,y) in pos.items()}


## 4  CodeGen → Blender .py script

In [38]:
def gen_blender_py(scene: dict, placements: dict, outfile='scene.py'):
    lines = ["import bpy"]
    for n in scene['nodes']:
        nid = n['id']
        asset = n['asset']
        x,y = placements[nid]
        z = n['size'][2]/2
        lines += [
            f'bpy.ops.wm.append(filename="{asset}")',
            f'bpy.context.selected_objects[0].location = ({x:.2f}, {y:.2f}, {z:.2f})'
        ]
    with open(outfile,'w') as f:
        f.write("\n".join(lines))
    return outfile

## 5  Headless Blender render

In [62]:
import subprocess, pathlib

def render_blend(pyfile: str, outpng: str='assets/renders/render.png'):
    blender_bin = '/Applications/Blender.app/Contents/MacOS/Blender'  # adjust path if needed
    cmd = [blender_bin, '-b', '--python', pyfile, '-o', outpng, '-f', '1']
    print(' '.join(cmd))
    subprocess.run(cmd, check=True)
    return pathlib.Path(outpng)

## 6  Vision‑language verifier (stub)

In [40]:
def verify_scene(image_path: str, scene: dict) -> float:
    """Return fraction of relations judged correct by GPT‑4V (stub)."""
    # TODO: implement with openai.Vision API once available.
    return 1.0  # assume perfect


## 7  End‑to‑end pipeline

In [65]:
def plan_and_render(prompt: str):
    scene = llm_to_scene(prompt)
    # print('scene', scene)
    scene_file = helper.save_scene_graph(scene)
    placements = solve_scene(scene)
    py_script = gen_blender_py(scene, placements)
    img = render_blend(py_script)
    score = verify_scene(img, scene)
    print(f'Verification score: {score}')
    return img

## 8  Example run

In [69]:
example_prompt = "Place a 2m sofa in front of a 1m coffee table with a 0.3m gap, inside a 4*3 m room."

plan_and_render(example_prompt)

response ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{\n  "nodes": [\n    {\n      "id": "1",\n      "size": [2.0, 1.0, 0.6],\n      "asset": "sofa"\n    },\n    {\n      "id": "2",\n      "size": [1.0, 0.5, 0.3],\n      "asset": "coffee_table"\n    },\n    {\n      "id": "3",\n      "size": [4.0, 3.0, 2.5],\n      "asset": "room"\n    }\n  ],\n  "constraints": [\n    {\n      "type": "front_of",\n      "a": "1",\n      "b": "2",\n      "gap": 0.3\n    }\n  ]\n}', name='create_scene_graph'), tool_calls=None)


AttributeError: module 'helper' has no attribute 'save_scene_graph'